# Use public data to replicate S.Chevrot (2000) *Earth and Planetary Science Letters* results Figure 3

In [ ]:
import os

import numpy as np
import pandas as pd

import rf
import rf.imaging

import matplotlib.pyplot as plt
import seaborn as sns

from obspy import UTCDateTime

In [ ]:
import seismic.receiver_fn.rf_util as rf_util
import seismic.receiver_fn.rf_plot_utils as rf_plot_utils
import seismic.receiver_fn.rf_stacking as rf_stacking

### Load the data file of processed RF traces for analysis

In [ ]:
station = 'SD02'
data = rf_util.read_h5_rf(r"..\DATA\7B_rfs_19930503T030058-19950810T012516_LQT_fd_qual.h5", root='/waveforms/7B.SD02.')
data

## Process data and present RF stacking

### Filter data down to only RF traces (filter out raw traces)

In [ ]:
data_rf = rf.RFStream([tr for tr in data if tr.stats.type == 'rf'])
del data
data_rf

### Check the main data channel code and set channel accordingly

In [ ]:
set([tr.stats.channel for tr in data_rf])

In [ ]:
channel = 'BHQ'

### Check distribution of SNR values to help us choose a cutoff

In [ ]:
snr_all = np.array([tr.stats.snr for tr in data_rf])

In [ ]:
plt.hist(snr_all, bins=20)
# plt.xlim((0, 20))
plt.show()

In [ ]:
# Pick a cutoff to get about 20 traces
cutoff_snr = 2.0

### Filter RFs to those with good enough SNR

In [ ]:
data_good = rf.RFStream(sorted([tr for tr in data_rf if tr.stats.snr >= cutoff_snr], key=lambda v: v.stats.distance))
len(data_good)

### Plot the good RFs

In [ ]:
time_window=(-1.0, 30.0)
trace_height=0.15
stack_height=0.6
scale=2

In [ ]:
_ = data_good.plot_rf(fillcolors=('#000000', '#a0a0a0'), trim=time_window, scale=scale, trace_height=trace_height, stack_height=stack_height)

In [ ]:
# Experimental: scale traces by phase-weighting. Since there is no moveout, this phase weighting dilutes the strength of the PpPs and higher multiples,
# so this does not help the stacked result.
# pw = rf_util.phase_weights(data_good)
# data_good_pw = data_good.copy()
# for tr in data_good_pw:
#     tr.data = tr.data*pw
# _ = data_good_pw.plot_rf(fillcolors=('#000000', '#a0a0a0'), trim=time_window, scale=scale, trace_height=trace_height, stack_height=stack_height)

## H-k stacking

In [ ]:
db = rf_util.rf_to_dict(data_good)
data_sta = db[station]

In [ ]:
weighting = (0.5, 0.5, 0.0)

k_grid, h_grid, hk_stack = rf_stacking.compute_hk_stack(data_sta, channel, h_range=np.linspace(20.0, 70.0, 501), root_order=2, V_p=6.4)

# Sum the phases
hk_stack_sum = rf_stacking.compute_weighted_stack(hk_stack, weighting)

# Raise the final sum over phases to power >1 to increase contrast
hk_stack_sum = rf_util.signed_nth_power(hk_stack_sum, 2)

sta = data_sta[channel][0].stats.station
num = len(data_sta[channel])
save_file = 'Hk_stack_SD02.BHQ_validation.png'
rf_plot_utils.plot_hk_stack(k_grid, h_grid, hk_stack_sum, title='Station ' + sta + '.{}'.format(channel), num=num, save_file=save_file)

## Further analysis to disambiguate which is the "correct" selection of *H-k* maximum

### Generate simple picks on $t_1$, $t_2$ and use analytic solution for $(H, \kappa)$ to compute scattergram based on picks

In [ ]:
# Use larger dataset for denser scattergram
adequate_cutoff = 1.5
data_adequate = rf.RFStream(sorted([tr for tr in data_rf if tr.stats.snr >= adequate_cutoff], key=lambda v: v.stats.distance))
len(data_adequate)

In [ ]:
# Get t1, t2 from picking maxima within narrow time bands based on RF plot above
t1 = []
t2 = []
incl = []
slowness_secperkm = []
km_per_deg = 111.1949
snr_ad = np.array([tr.stats.snr for tr in data_adequate])
for tr in data_adequate:
    t_offset = tr.stats.onset - tr.stats.starttime
    t_rel = tr.times() - t_offset
    t1_mask = ((t_rel >= 4) & (t_rel <= 7.5))
    t2_mask = ((t_rel >= 15) & (t_rel <= 20))
    t1_max = np.max(tr.data[t1_mask])
    t2_max = np.max(tr.data[t2_mask])
    t1_index = np.where(tr.data[t1_mask] == t1_max)
    t2_index = np.where(tr.data[t2_mask] == t2_max)
    t1_val = t_rel[t1_mask][t1_index]
    t2_val = t_rel[t2_mask][t2_index]
    t1.append(t1_val.mean())
    t2.append(t2_val.mean())
    incl.append(tr.stats.inclination*np.pi/180.0)
    slowness_secperkm.append(tr.stats.slowness/km_per_deg)
t1 = np.array(t1)
t2 = np.array(t2)
incl = np.array(incl)
slowness_secperkm = np.array(slowness_secperkm)
ray_param = np.sin(incl)*slowness_secperkm

In [ ]:
alpha = t2 - t1
beta = t1 + t2
V_p = 6.4

In [ ]:
p = ray_param
H = alpha/(2*np.sqrt(1/V_p**2 - p*p))
k = V_p*np.sqrt((beta/(2*H))**2 + p*p)

In [ ]:
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

plt.figure(figsize=(13,12))
ax = plt.gca()
sns.scatterplot(k, H, hue=snr_ad, size=snr_ad, sizes=(50, 250), s=1000, alpha=0.9, ax=ax)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel(r'$\kappa = \frac{V_p}{V_s}$ (ratio)', fontsize=14)
plt.ylabel('H = Moho depth (km)', fontsize=14)
plt.grid(color="#80808080", linestyle=":")
plt.xlim(1.3, 2.1)
plt.ylim(20, 70)
plt.legend(title='SNR')
plt.title('Per-event $H-\kappa$ scattergram based on $t_1$, $t_2$ picks', fontsize=20)
# ellipse = Ellipse((0, 0), width=0.4, height=5, facecolor=None, edgecolor='#202020', linestyle='--', fill=False)
# transf = transforms.Affine2D().rotate_deg(0).translate(1.85, 39)
# ellipse.set_transform(transf + ax.transData)
# ax.add_patch(ellipse)
plt.savefig('Hk_scattergram_SD02.BHQ_validation.png', dpi=300)
plt.show()

## APPENDIX: Derivation of solution for $(H, \kappa)$ from $(t_1, t_2)$

There are various similar and equivalent formulations for the theoretical arrival times $t_1, t_2$ of the Ps and PpPs phases respectively. The formulation chosen here is that of Youlin Chen *et al.*, "Crustal structure beneath China from receiver function analysis", *Journal of Geophysical Research*, Vol.115, B03307 (2010), doi:10.1029/2009JB006386.

Starting with:

$t_1 = H \left[\sqrt{\left(\frac{\kappa}{V_p}\right)^2 - p^2} - \sqrt{\left(\frac{1}{V_p}\right)^2 - p^2} \right] \qquad \text{Ps}$

and

$t_2 = H \left[\sqrt{\left(\frac{\kappa}{V_p}\right)^2 - p^2} + \sqrt{\left(\frac{1}{V_p}\right)^2 - p^2} \right] \qquad \text{PpPs}$

where $H$ = depth to Moho, $\kappa = \frac{V_p}{V_s}$, $V_p$ = P-wave velocity, $p$ = ray parameter (per ray/event).

The let $\alpha = t_2 - t_1$, $\beta = t_1 + t_2$. Algebraic solution yields

$\alpha = 2H\sqrt{\left(\frac{1}{V_p}\right)^2 - p^2}$

$\implies H = \frac{\alpha}{2\sqrt{\left(\frac{1}{V_p}\right)^2 - p^2}}$

in which one can see that $\alpha$ is independent of $\kappa$ and depends only on $H$.

For $\beta$,

$\beta = 2H\sqrt{\left(\frac{\kappa}{V_p}\right)^2 - p^2}$

$\implies \kappa = \frac{V_p}{V_s} = V_p \sqrt{\left(\frac{\beta}{2H}\right)^2 + p^2}$